In [2]:
# Set up the configuration for accessing the storage account
storage_account_name = "team12storage"
storage_account_key = "ZdpCqxRwuQbJ8fUYN7EMbTLjOyj+I9rI6IjcQ/uU+4aFG46TQqjiz1Piq9SWKGkkjszo3r94l26M+AStR5PvPg=="

spark.conf.set(
    "fs.azure.account.key." + storage_account_name + ".dfs.core.windows.net",
    storage_account_key
)

StatementMeta(StockPool, 5, 3, Finished, Available, Finished)

In [1]:
container = "team12blobcontainer"

StatementMeta(StockPool, 5, 2, Finished, Available, Finished)

In [3]:
abfss_base_path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/"
df = spark.read.csv(f"{abfss_base_path}Medallion/Bronze/Stock_Volume/batch/part-00000-7a861d5b-ba71-4207-b368-5144fc367747-c000.csv", inferSchema=True)

StatementMeta(StockPool, 5, 4, Finished, Available, Finished)

In [4]:
df = df.toPandas()


StatementMeta(StockPool, 5, 5, Finished, Available, Finished)

In [6]:
!pip install yfinance --upgrade --quiet

StatementMeta(StockPool, 5, 7, Finished, Available, Finished)

In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

StatementMeta(StockPool, 5, 8, Finished, Available, Finished)

In [8]:
df

StatementMeta(StockPool, 5, 9, Finished, Available, Finished)

,_c0,_c1,_c2,_c3,_c4,_c5
0,2024-01-02,48.149918,49.276493,47.577135,49.225514,411254000
1,2024-01-03,47.551144,48.165911,47.302237,47.467176,320896000
2,2024-01-04,47.979984,48.481795,47.490167,47.749068,306535000
3,2024-01-05,49.078568,49.528399,48.287864,48.443808,415039000
4,2024-01-08,52.233383,52.255378,49.460426,49.493415,642510000
...,...,...,...,...,...,...
326,2025-04-22,98.889999,99.809998,97.279999,98.779999,241004800
327,2025-04-23,102.709999,104.800003,102.019997,104.519997,247526000
328,2025-04-24,106.430000,106.540001,103.110001,103.480003,220815000
329,2025-04-25,111.010002,111.919998,105.730003,106.849998,251064700


In [9]:
# Rename columns
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

# Convert Date column to datetime (if not already)
df['Date'] = pd.to_datetime(df['Date'])

# Sort by Date to ensure chronological order
df.sort_values('Date', inplace=True)

# Check for and drop any duplicate rows
df.drop_duplicates(inplace=True)

# Print summary to validate
print(df.info())
print(df.head())


StatementMeta(StockPool, 5, 10, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331 entries, 0 to 330
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    331 non-null    datetime64[ns]
 1   Close   331 non-null    float64       
 2   High    331 non-null    float64       
 3   Low     331 non-null    float64       
 4   Open    331 non-null    float64       
 5   Volume  331 non-null    int32         
dtypes: datetime64[ns](1), float64(4), int32(1)
memory usage: 16.8 KB
None
        Date      Close       High        Low       Open     Volume
0 2024-01-02  48.149918  49.276493  47.577135  49.225514  411254000
1 2024-01-03  47.551144  48.165911  47.302237  47.467176  320896000
2 2024-01-04  47.979984  48.481795  47.490167  47.749068  306535000
3 2024-01-05  49.078568  49.528399  48.287864  48.443808  415039000
4 2024-01-08  52.233383  52.255378  49.460426  49.493415  642510000


In [10]:
# Define the output path with specific folder/file name structure
output_path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/Medallion/Silver/Stock_Volume_Silver/"

# Convert pandas timestamp to string format that Spark can handle
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Create spark dataframe directly from pandas
spark_df = spark.createDataFrame(df)

# Write with controlled partitioning to ensure all data is included
spark_df.repartition(1).write.format("parquet").mode("overwrite").save(output_path)

StatementMeta(StockPool, 5, 11, Finished, Available, Finished)